In [1]:
!pip install selenium --upgrade --quiet
!pip install beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 31.7 MB/s eta 0:00:00


In [2]:
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
!apt-get update --quiet
!apt-get install chromium chromium-driver --quiet
!apt-get install chromium -y
!apt-get install -y gconf-service libasound2 libatk1.0-0 libcairo2 libcups2 libfontconfig1 libgdk-pixbuf2.0-0 libgtk-3-0 libnspr4 libpango-1.0-0 libxss1 fonts-liberation libappindicator1 libnss3 lsb-release xdg-utils
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb; apt-get -fy install

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,611 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,458 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,4

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
import concurrent.futures
import random
import json


In [5]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [6]:
# fatching Villages List.
def get_village(url):
    village_list = {}
    driver = web_driver()
    driver.get(url)
    delay = random.uniform(1, 3)
    time.sleep(delay)

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('tbody')
    if table:
        rows = table.find_all('tr')
        # Fatching on 3 villages name
        for row in rows[:3]:
            cols = row.find_all('td')
            if len(cols) >= 2:
                village_name = cols[1].text.strip()
                village_code = cols[2].text.strip()
                village_list[village_name] = village_code
    return village_list


# Fatching Tehsil names
def get_tehsil(url):
  tehsil_list = {}
  driver = web_driver()
  driver.get(url)
  delay = random.uniform(2, 3)
  time.sleep(delay)

  # Get the page source after it's loaded
  page_source = driver.page_source

  # Parse the page source with BeautifulSoup
  soup = BeautifulSoup(page_source, 'html.parser')
  table = soup.find_all('tbody')

  #Extracting only 3 tehsil names
  for tehsils in table[:3]:
    tehsil = tehsils.find_all('a')
    for all_tehsil in tehsil[:3]:
      tehsil_name = all_tehsil.text
      tehsil_list[tehsil_name] = {}
      villages_url = 'https://vlist.in' + all_tehsil['href']
      delay = random.uniform(2, 4)
      time.sleep(delay)
      villages = get_village(villages_url)
      tehsil_list[tehsil_name] = villages
  return tehsil_list


# Fatching District Names
def get_district(url):
  distict_list = {}
  driver = web_driver()
  driver.get(url)
  delay = random.uniform(2, 3)
  time.sleep(delay)
  page_source = driver.page_source
  soup = BeautifulSoup(page_source, 'html.parser')
  table = soup.find_all('tbody')

  # Extracting only 3 disticts names
  for disticts in table[:3]:
    state = disticts.find_all('a')
    for all_distict in state[:3]:
      delay = random.uniform(2,5)
      time.sleep(delay)
      distict_name = all_distict.text
      distict_list[distict_name] = {}
      tehsil_url = 'https://vlist.in' + all_distict['href']
      delay = random.uniform(1, 3)
      time.sleep(delay)
      distict = get_tehsil(tehsil_url)
      distict_list[distict_name] = distict
  return distict_list


# Fatching States Name
def get_one_state(url):
    try:
        with open('states_data.json', 'r') as json_file:
            states_list = json.load(json_file)
    except FileNotFoundError:
        states_list = {}
    driver = web_driver()
    try:
        driver.get(url)
        delay = random.uniform(2, 5)
        time.sleep(delay)

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table = soup.find_all('tbody')

        # Check if table is found and extract the first state URL
        if table:
            for new in table:
                state_rows = new.find_all('tr')

                for i in range(1, 36):  # Loop from 1 to 35
                    if i < len(state_rows):  # Ensure the index is within range
                        state_row = state_rows[i]
                        state_link = state_row.find('a')
                        if state_link:
                          state_name = state_link.text
                          states_list[state_name] = {}
                          district_url = 'https://vlist.in' + state_link['href']
                          district_data = get_district(district_url)
                          delay = random.uniform(3, 5)
                          time.sleep(delay)
                        states_list[state_name] = district_data

            # Save the states_list dictionary to a JSON file
            with open('states_data.json', 'w') as json_file:
                json.dump(states_list, json_file, indent=4)
    finally:
        driver.quit()  # Make sure to quit the driver after use

    return states_list


url = 'https://vlist.in/'
print(get_one_state(url))

{'Andaman and Nicobar Islands': {'Nicobars': {'Car Nicobar': {'Arong': '645015', 'Big Lapati': '645025'}, 'Great Nicobar': {'7 km Farm': '645196', 'Afra Bay': '645166'}, 'Nancowry': {'Al-Hit-Touch/Balu Basti': '645094', 'Alhiat': '645030'}}, 'North  & Middle Andaman': {'Diglipur': {'Aam Tikry (FDCA)': '645280', 'Aerial Bay (RV)': '645227'}, 'Mayabunder': {'Asha Nagar (EFA)': '645338', 'Aves Island (RV)': '645293'}, 'Rangat': {'Abaygarh (RV)': '645380', 'Adojig (RV) (including EFA)': '645374'}}, 'South Andaman': {'Ferrargunj': {'Alipur (RV)': '645448', 'Aniket (RV)': '645464'}, 'Little Andaman': {'Butler Bay Forest Camp 4-I (FDCA)': '645570', 'Butler Bay Forest Camp 4-II (FDCA)': '645569'}, 'Port Blair': {'Bada Khari (FC)': '645547', 'Bada Nallaha/Bada Balu (EFA)': '645542'}}}, 'Andhra Pradesh': {'Adilabad': {'Adilabad': {'Adilabad (M)': '802896', 'Ankapoor': '569005'}, 'Asifabad ': {'Ada': '569480', 'Ada - Dasnapur': '569522'}, 'Bazarhathnoor': {'Ananthapur': '569647', 'Balanpur': '569